## A demo notebook to publish datacubes and workflow to EarthCODE catalog
### A DeepESDL example notebook

Please, also refer to the [DeepESDL documentation](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/) and visit the platform's [website](https://www.earthsystemdatalab.net/) for further information!

Brockmann Consult, 2025

-----------------

**This notebook runs with the python environment `users-deep-code-test`, please checkout the documentation for [help on changing the environment](https://deepesdl.readthedocs.io/en/latest/guide/jupyterlab/#python-environment-selection-of-the-jupyter-kerne).**

###  📘 Pre-requisite:
Before using the deep-code CLI or API to publish metadata, users must configure GitHub access by creating a .gitaccess file in the working directory from which deep-code is executed.

1. Generate a Personal Access Token (PAT) from your GitHUB account:
    1. Navigate to GitHub → Settings → Developer settings → Personal access tokens.
    2. Click “Generate new token”.
    3. Choose the following scopes to ensure full access:
        - repo (Full control of repositories — includes fork, pull, push, and read)
    4. Generate the token and copy it immediately — GitHub won’t show it again.

2. Create a .gitaccess File

In the same directory where you run the deep-code commands, create a file named .gitaccess with the following content:
```
github-username: your-git-user
github-token: personal access token
```
Replace your-git-user and your-personal-access-token with your actual GitHub username and token.

This file is required to allow deep-code to fork the Open Science Metadata repository, commit metadata changes, and open a pull request to the EarthCODE Catalog.

In [1]:
import os
import warnings

from xcube.core.store import new_data_store

from deep_code.tools.lint import LintDataset
from deep_code.tools.publish import Publisher

In [2]:
warnings.filterwarnings('ignore')

## Generate starter configuration templates for publishing to EarthCODE openscience catalog.

In [ ]:
!deep-code generate-config

## Here we create a small dataset from xcube-cmems store

In [2]:
store = new_data_store("cmems")
store

WARNING - 2025-09-23T14:37:42Z - COPERNICUS_MARINE_SERVICE_USERNAME is deprecated. Please use COPERNICUSMARINE_SERVICE_USERNAME instead.
WARNING - 2025-09-23T14:37:42Z - COPERNICUS_MARINE_SERVICE_PASSWORD is deprecated. Please use COPERNICUSMARINE_SERVICE_PASSWORD instead.
/home/tejas/micromamba/envs/deep-code/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = store.open_data(
    "DMI-BALTIC-SST-L3S-NRT-OBS_FULL_TIME_SERIE",
    variable_names=["sea_surface_temperature"],
    bbox=[9, 53, 20, 62],
    time_range=("2022-01-01", "2022-01-05"),
)
ds

INFO - 2025-09-23T14:37:44Z - Selected dataset version: "201904"
INFO:copernicusmarine:Selected dataset version: "201904"
INFO - 2025-09-23T14:37:44Z - Selected dataset part: "default"
INFO:copernicusmarine:Selected dataset part: "default"


<xarray.Dataset> Size: 10MB
Dimensions:                  (time: 5, latitude: 451, longitude: 551)
Coordinates:
  * latitude                 (latitude) float32 2kB 53.0 53.02 ... 61.98 62.0
  * longitude                (longitude) float32 2kB 9.0 9.02 ... 19.98 20.0
  * time                     (time) datetime64[ns] 40B 2022-01-01 ... 2022-01-05
Data variables:
    sea_surface_temperature  (time, latitude, longitude) float64 10MB dask.array<chunksize=(1, 451, 512), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        DMI North Sea/Baltic Sea SST, L3S, (SST_BAL_SST_L3S_NRT_OBS...
    source:       VIIRS_SST_NPP_NAR-OSISAF-L3C, SEVIRI_SST-OSISAF-L3C, EUR-L2...
    history:      Version 1.0
    institution:  Danish Meteorological Institute, DMI
    references:   Høyer, J. L., Le Borgne, P., & Eastwood, S. (2014). A bias ...
    comment:      IN NO EVENT SHALL DMI OR ITS REPRESENTATIVES BE LIABLE FOR ...

## Lint your in-memory dataset for metadata correctness and completness, before publishing to EarthCODE open science catalog

In [5]:
linter = LintDataset(dataset=ds)
linter.lint_dataset()

ds.coords['latitude'],warn,"Unexpected encoding '_FillValue', coordinates must not have missing data.",var-missing-data
ds.coords['longitude'],warn,"Unexpected encoding '_FillValue', coordinates must not have missing data.",var-missing-data
ds.data_vars['sea_surface_temperature'],warn,Valid ranges are not recognized by xarray (as of Feb 2025).,var-missing-data
ds.data_vars['sea_surface_temperature'],error,Variable 'sea_surface_temperature' missing 'gcmd_keyword_url' attribute.,deepcode/variable-gcmd-keyword-url
ds,error,Dataset missing required 'description' attribute.,deepcode/dataset-description


## Fix the errors from the linter

Adding gcmd_keyword_url connects your data to a semantic network of Earth science concepts, enabling:

- Better automated discovery

- Stronger metadata interoperability

- Alignment with international FAIR standards

To find the the gcmd url for your variable, please use, https://gcmd.earthdata.nasa.gov/KeywordViewer/scheme/all?gtm_scheme=all

In [6]:
ds.attrs["description"] = (
    "This is a extracted dataset from copernicus marine data store" 
)

ds["sea_surface_temperature"].attrs["gcmd_keyword_url"] = "https://gcmd.earthdata.nasa.gov/KeywordViewer/scheme/all/e4d58a7f-7eaa-4f75-996a-18238c698063?gtm_keyword=SEA%20SURFACE%20FOUNDATION%20TEMPERATURE&gtm_scheme=Earth%20Science"

## Write the dataset to the team s3 bucket

In [ ]:
S3_USER_STORAGE_KEY = os.environ["S3_USER_STORAGE_KEY"]
S3_USER_STORAGE_SECRET = os.environ["S3_USER_STORAGE_SECRET"]
S3_USER_STORAGE_BUCKET = os.environ["S3_USER_STORAGE_BUCKET"]

In [ ]:
team_store = new_data_store(
    "s3", 
    root=S3_USER_STORAGE_BUCKET, 
    storage_options=dict(
        anon=False, 
        key=S3_USER_STORAGE_KEY, 
        secret=S3_USER_STORAGE_SECRET
    )
)

In [ ]:
team_store.write_data(ds, "cmems_sst_v2.zarr", replace=True)

The user workflow which is the JNB has to be pushed to git repository: https://github.com/deepesdl/cube-gen/blob/main/Permafrost/Create-CCI-Permafrost-cube-EarthCODE.ipynb

# 📘 Publishing Metadata to the EarthCODE Catalogue

Once the dataset and workflow metadata are prepared and validated, users can initiate the publishing process using the deep-code CLI or the python function. The following command automates the entire workflow:

## 🔹 The below command performs the following steps:

1. Generates valid STAC and OGC API Records based on the provided configuration files

2. Forks the open-science-catalog-metadata repository on GitHub

3. Inserts the generated records into the correct directory structure

4. Creates a Pull Request (PR) for review by the Open Science Catalog steward

Execute the below cell to confirm if you are in the correct directory, the current directory should have the .gitaccess file

In [2]:
notebook_path = os.path.abspath("publish_to_EarthCODE.ipynb")
notebook_dir = os.path.dirname(notebook_path)
print("Current Working Directory:", notebook_dir)
# Change working directory
os.chdir(notebook_dir)

# Confirm
print("Current Working Directory:", os.getcwd())

Current Working Directory: /home/tejas/bc/projects/deepesdl/deep-code/examples/notebooks
Current Working Directory: /home/tejas/bc/projects/deepesdl/deep-code/examples/notebooks


## publish using the python function
Before Publish, Please prepare dataset and/or workflow configuration file for your use case

In [3]:
# publish dataset and workflow using the python function
publisher = Publisher(
    dataset_config_path="dataset-config.yaml",
    workflow_config_path="workflow-config.yaml"
)
publisher.publish()

# # To publish only workflow
# publisher = Publisher(
#     workflow_config_path="workflow-config.yaml"
# )
# publisher.publish(mode="worflow")

# # To publish only dataset
# publisher = Publisher(
#     dataset_config_path="dataset-config.yaml",
# )
# publisher.publish(mode="dataset")

INFO:root:Forking repository...
INFO:root:Repository forked to tejasmharish/open-science-catalog-metadata-staging
INFO:root:Checking local repository...
INFO:root:Cloning forked repository...
Cloning into '/home/tejas/temp_repo'...
INFO:root:Repository cloned to /home/tejas/temp_repo
INFO:deep_code.tools.publish:Generating STAC collection...
INFO:deep_code.utils.dataset_stac_generator:Attempting to open dataset 'cmems_sst_v2.zarr' with configuration: Public store
INFO:deep_code.utils.dataset_stac_generator:Successfully opened dataset 'cmems_sst_v2.zarr' with configuration: Public store
INFO:deep_code.tools.publish:Variable catalog already exists for sea-surface-temperature, adding product link.
INFO:deep_code.tools.publish:Generating OGC API Record for the workflow...
INFO:root:Creating new branch: add-new-collection-cmems-sst-20250924111300...
Switched to a new branch 'add-new-collection-cmems-sst-20250924111300'
INFO:deep_code.tools.publish:Adding products/cmems-sst/collection.json t

[add-new-collection-cmems-sst-20250924111300 5093f608] Publish dataset: cmems-sst + workflow/experiment: demo-cmems-sst-workflow
 9 files changed, 434 insertions(+), 6 deletions(-)
 create mode 100644 experiments/demo-cmems-sst-workflow/record.json
 create mode 100644 products/cmems-sst/collection.json
 create mode 100644 workflows/demo-cmems-sst-workflow/record.json


remote: 
remote: Create a pull request for 'add-new-collection-cmems-sst-20250924111300' on GitHub by visiting:        
remote:      https://github.com/tejasmharish/open-science-catalog-metadata-staging/pull/new/add-new-collection-cmems-sst-20250924111300        
remote: 
To https://github.com/tejasmharish/open-science-catalog-metadata-staging.git
 * [new branch]        add-new-collection-cmems-sst-20250924111300 -> add-new-collection-cmems-sst-20250924111300
INFO:root:Creating a pull request...


Branch 'add-new-collection-cmems-sst-20250924111300' set up to track remote branch 'add-new-collection-cmems-sst-20250924111300' from 'origin'.


INFO:root:Pull request created: https://github.com/ESA-EarthCODE/open-science-catalog-metadata-staging/pull/162
INFO:deep_code.tools.publish:Pull request created: None
INFO:root:Cleaning up local repository...
INFO:deep_code.tools.publish:Pull request created: None


## publish using cli
- Publisher now supports `mode` parameter, This allows more flexible publishing:
  - `"dataset"` → publish dataset only
  - `"workflow"` → publish workflow only
  - `"all"` → publish both (default)

In [ ]:
!deep-code publish dataset-config.yaml workflow-config.yaml 

publish dataset with -m dataset (dataset mode)

In [ ]:
!deep-code publish dataset-config.yaml -m dataset

publish workflow with -m workflow (workflow mode)

In [ ]:
!deep-code publish workflow-config.yaml -m workflow

explicitly pass workflow config

In [ ]:
!deep-code publish --workflow-config workflow-config.yaml 